A good fit for approaching these concepts is the Titanic Survival dataset: a widely known starter-level dataset for classification.

We will use XGBoost because:
- efficient implementation of Gradient Boosting, a very powerful algorithm for classification and regression
- embedded regularization to avoid overfitting
- rich of features for tune details

Pretty much a silver bullet.

The goal of this example is not to achieve the best possible AUC for Titanic Survival. Rather, it's to build a robust model that is likely to generalize well. 


Remember that **the ultimate goal when building an ML system is to learn patterns that will apply to new unseen data**. We train and assess models on a small slice of the world (our beloved dataset), but they only deliver value to stakeholder and customers once running in production.

Ideally, we want to analyze how well our model can predict the survival outcome of a generic ship disaster. In order to simulate this situation, we slightly modifiy the test set available from Kaggle. We shuffle the columns containing personal information of each passenger (PassengerId, Name, Cabin, Ticket) - *this is far from perfect, but it works for the sake of this example.*

In [1]:
%matplotlib inline

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.titanic import TitanicSurvival

titanic_survival = TitanicSurvival('/home/flavio/data/Titanic/train.csv')
titanic_survival

ModuleNotFoundError: No module named 'code.titanic'; 'code' is not a package

In [ ]:
titanic_survival.X.head()

We need to apply quite standard preprocessing:
- impute missing values
- encode categorical values to numeric format

That's the result:

In [ ]:
titanic_survival.preprocess().head()

Let's go autopilot and fit an XGBoost to the data. We perform a quick hyperparameter search to be sure the hyperparameter setup makes sense with our data.

In [ ]:
titanic_survival.fit()

Just to have an idea, how well does the model perform on the test set?

In [ ]:
titanic_survival.evaluate_test()

Although not bad, the AUC is significantly less than on the train set. It may mean that the model overfitted on those features containing personal information, which we shuffled on the test set.

## How does the model rely on each feature?

The first, easier and most common analysis on a trained model is often call _feature importances_. It aims to measure the strenght of the model dependence, or reliance, on each feature.

XGBoost has a built-in function to plot feature importances. It supports three algorithms, all specific of tree ensemble models. According to the documentation:

> - ”weight” is the number of times a feature appears in a tree
> - ”gain” is the average gain of splits which use the feature
> - ”cover” is the average coverage of splits which use the feature where coverage is defined as the number of samples  affected by the split

Let's try them out.

In [ ]:
titanic_survival.feature_importances('weight')

In [ ]:
titanic_survival.feature_importances('gain')

In [ ]:
titanic_survival.feature_importances('cover')

Ok, confusing. `cover` and `gain` seems somewhat in accordance, with `weight` telling a whole different story. 

Fortunately, there is an alternative. Inspired by the original paper on Random Forest by Breiman, this model agnostic method is often called `permutation importance`. An open-source implementation is found in `eli5`, which also provides [a clear explanation on how it works](https://eli5.readthedocs.io/en/latest/blackbox/permutation_importance.html).

Let's try it out:

In [ ]:
titanic_survival.permutation_importances()

That makes more sense, right? 

We know for a fact that they applied the code ["women and children first"](https://www.newscientist.com/article/dn22119-sinking-the-titanic-women-and-children-first-myth/). `Sex` is the most importance feature, with `Age` being in the top three. The knowledge we have about the reality seems to match what the model learned.

Partial Dependence Plots show the relationship between a feature and the predicted outcome of the model. You can learn more about Partial Dependence Plots here [link to medium].

We can draw a PDP to be sure that the model learned the same pattern:

In [ ]:
titanic_survival.show_pdp('Sex')

Cool! We verified that observations with Sex equal to female are associated with a far higher probability of survival.

The second most important feature is `Pclass`, the ticket class. The intuitive sense suggests that wealthier passengers have more likelihood of getting a seat in a lifeboat.

Let's find out if the model matches our intuition:

In [ ]:
titanic_survival.show_pdp('Pclass')

Name??

In [ ]:
titanic_survival.show_pdp('Name')

Weird, don't want that

In [ ]:
titanic_survival.drop_feature('Name')

In [ ]:
titanic_survival.fit()

In [ ]:
titanic_survival.permutation_importances()

Oops, now Ticket 

In [ ]:
titanic_survival.drop_feature('Ticket')

In [ ]:
titanic_survival.fit()

Pretty much same AUC without the most important feature??

Remember: that feature was important for **that specific model**, not for all possible models. It's not a concept related to the dataset, but to a model obtained with certain design decisions.

In [ ]:
titanic_survival.permutation_importances()

In [ ]:
titanic_survival.drop_feature(['Cabin', 'PassengerId'])

In [ ]:
titanic_survival.fit()

In [ ]:
titanic_survival.permutation_importances()

Better, now Fare and Age are more generalizable. How exactly do they influence output?

In [ ]:
titanic_survival.show_pdp('Fare')

Monotonic is easier to explain:

In [ ]:
titanic_survival.force_monotonicity('Fare', 1)
titanic_survival.fit()

In [ ]:
titanic_survival.show_pdp('Fare')

In [ ]:
titanic_survival.permutation_importances()

Very interesting. Fare is still important, but relatively less. 

In [ ]:
titanic_survival.evaluate_test()